In [ ]:
# !pip3 install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
# !pip install optuna

In [90]:
#Load libraries
import time
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib
import pandas as pd

In [58]:
#checking for device
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


## Augmentation object creation

In [59]:
#Transforms
transformer=transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

## Data loading

In [60]:
#Dataloader

#Path for training and testing directory
train_path='C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/train'
test_path='C:/Users/user/Documents/GitHub/GroceryCV/GroceryStoreDataset-master/dataset/test'

train_loader=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=64, shuffle=True
)
test_loader=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

In [61]:
root=pathlib.Path(train_path)
classes=sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [62]:
print(classes)

['Apple', 'Asparagus', 'Aubergine', 'Avocado', 'Banana', 'Cabbage', 'Carrots', 'Cucumber', 'Garlic', 'Ginger', 'Juice', 'Kiwi', 'Leek', 'Lemon', 'Lime', 'Mango', 'Milk', 'Mushroom', 'Nectarine', 'Onion', 'Orange', 'Papaya', 'Passion-Fruit', 'Peach', 'Pear', 'Pepper', 'Pineapple', 'Plum', 'Potato', 'Sour-Milk', 'Soy-Milk', 'Tomato', 'Yoghurt', 'Zucchini']


## Build model

In [75]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=len(classes)):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        
        
        #Feed foward function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
            
            #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
            
        output=self.fc(output)
            
        return output

## Final Model evaluation

In [81]:
model=ConvNet(num_classes=len(classes)).to(device)

In [86]:
#Optmizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.001) 
loss_function=nn.CrossEntropyLoss()
num_epochs=10

In [87]:
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [88]:
print(train_count,test_count)

1239 165


In [89]:
start = time.time()
for epoch in range(num_epochs):
    
    #Evaluation and training on training dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
end = time.time()
print("Time taken:", end - start)

Epoch: 0 Train Loss: tensor(5.8442) Train Accuracy: 0.6957223567393059 Test Accuracy: 0.36363636363636365
Epoch: 1 Train Loss: tensor(3.5038) Train Accuracy: 0.7562550443906376 Test Accuracy: 0.36363636363636365
Epoch: 2 Train Loss: tensor(2.4093) Train Accuracy: 0.8272800645682001 Test Accuracy: 0.47878787878787876
Epoch: 3 Train Loss: tensor(1.7898) Train Accuracy: 0.8603712671509282 Test Accuracy: 0.4484848484848485
Epoch: 4 Train Loss: tensor(1.5784) Train Accuracy: 0.864406779661017 Test Accuracy: 0.49696969696969695
Epoch: 5 Train Loss: tensor(2.6369) Train Accuracy: 0.8184019370460048 Test Accuracy: 0.38181818181818183
Epoch: 6 Train Loss: tensor(1.4897) Train Accuracy: 0.8514931396287329 Test Accuracy: 0.45454545454545453
Epoch: 7 Train Loss: tensor(1.8725) Train Accuracy: 0.8426150121065376 Test Accuracy: 0.45454545454545453
Epoch: 8 Train Loss: tensor(1.1290) Train Accuracy: 0.8853914447134786 Test Accuracy: 0.47878787878787876
Epoch: 9 Train Loss: tensor(1.2279) Train Accura

## Model tuning
##### https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0

In [103]:
model=ConvNet(num_classes=len(classes)).to(device)

learning_rate_params = [0.1, 0.01, 0.001, 0.0001]
weight_decay_params = [0.1, 0.01, 0.001, 0.0001]

for x in learning_rate_params:
    for j in weight_decay_params:
        optimizer=Adam(model.parameters(),lr=x,weight_decay=j) 
        loss_function=nn.CrossEntropyLoss()
        num_epochs=10
        start = time.time()
        for epoch in range(num_epochs):

            #Evaluation and training on training dataset
            model.train()
            train_accuracy=0.0
            train_loss=0.0

            for i, (images,labels) in enumerate(train_loader):
                if torch.cuda.is_available():
                    images=Variable(images.cuda())
                    labels=Variable(labels.cuda())

                optimizer.zero_grad()

                outputs=model(images)
                loss=loss_function(outputs,labels)
                loss.backward()
                optimizer.step()


                train_loss+= loss.cpu().data*images.size(0)
                _,prediction=torch.max(outputs.data,1)

                train_accuracy+=int(torch.sum(prediction==labels.data))

            train_accuracy=train_accuracy/train_count
            train_loss=train_loss/train_count


            # Evaluation on testing dataset
            model.eval()

            test_accuracy=0.0
            for i, (images,labels) in enumerate(test_loader):
                if torch.cuda.is_available():
                    images=Variable(images.cuda())
                    labels=Variable(labels.cuda())

                outputs=model(images)
                _,prediction=torch.max(outputs.data,1)
                test_accuracy+=int(torch.sum(prediction==labels.data))

            test_accuracy=test_accuracy/test_count
            best_test_accuracy = 0 
            if test_accuracy > best_test_accuracy:
                best_test_accuracy = test_accuracy
        end = time.time()
        temp_df = pd.DataFrame({
            "accuracy":[best_test_accuracy],
            "model_training_time":[end - start],
            "learning_rate": [x],
            "weight_decay":[j]
        
        })
        print('Accuracy: '+str(best_test_accuracy)+'Model_training_time: '+str(end - start)+'Learning_rate: '+str(x)+' Weight_decay: '+str(j))

Accuracy: 0.06060606060606061Model_training_time: 78.63084483146667Learning_rate: 0.1 Weight_decay: 0.1
Accuracy: 0.06060606060606061Model_training_time: 67.68916320800781Learning_rate: 0.1 Weight_decay: 0.01
Accuracy: 0.06060606060606061Model_training_time: 64.28623294830322Learning_rate: 0.1 Weight_decay: 0.001
Accuracy: 0.06060606060606061Model_training_time: 64.16207265853882Learning_rate: 0.1 Weight_decay: 0.0001
Accuracy: 0.024242424242424242Model_training_time: 64.36404061317444Learning_rate: 0.01 Weight_decay: 0.1
Accuracy: 0.3393939393939394Model_training_time: 66.21559810638428Learning_rate: 0.01 Weight_decay: 0.01
Accuracy: 0.40606060606060607Model_training_time: 64.43703746795654Learning_rate: 0.01 Weight_decay: 0.001
Accuracy: 0.4303030303030303Model_training_time: 64.25464534759521Learning_rate: 0.01 Weight_decay: 0.0001
Accuracy: 0.45454545454545453Model_training_time: 64.52485823631287Learning_rate: 0.001 Weight_decay: 0.1
Accuracy: 0.47878787878787876Model_training_tim